<center><strong>Sistem Temu Kembali Informasi</strong><br />
<strong><font color="blue">Semester Gasal T.A. 2020/2021</font></strong><br />
</center>

<strong>Outline pertemuan minggu ke-4</strong><br />
<li> Word Embedding </li>

### Word Embedding

<p> Representasi dari teks ke dalam dense matriks. Term yang memiliki makna serupa berada pada jarak yang berdekatan pada ruang vektor </p>
<p> VSM merepresentasikan teks ke dalam sparse matriks </p>
<p> Sparse matriks memiliki ukuran yang besar dan memiliki banyak nilai 0 </p>
<p> Dense matriks ukurannya lebih kecil, dapat diinisialisasi, dan tidak memiliki nilai 0 </p>
<h2><img alt="" src="figures/3_word_embeddings.png" style="height: 296px ; width: 602px" /></h2>

#### Algoritma word embedding
<ul>
    <li> Skip-gram </li>
    <li> CBoW (Continuous Bag of Words)</li>
    <li> GloVe (Global Vector)</li>
    <li> FastText </li>
    <li> BERT (based on context) </li>
</ul>

#### Skip Gram dan CBoW
<h2><img alt="" src="figures/sg-cbow.png"/></h2>

<p> Dimana w(t) merupakan kata target dan w(t-n) - w(t+n) adalah konteks (kata yang berada disekitar kata target. n merupakan ukuran dari window size. Apabila n = 2 maka konteks didefinisikan sebagai 2 kata sebelum dan setelah kata target.</p>
<p> Contoh: "the big brown bear and the fox"
    <br>Apabila kata target adalah = "brown" dan n = 2 
    <br>Maka: 
    <br>w(t) = "brown"
    <br>w(t-2) = "the"
    <br>w(t-1) = "big"
    <br>w(t+1) = "bear"
    <br>w(t+2) = "and"
</p>
    

In [1]:
# download 3 categoies from 20newgroup dataset
# save it as binary

import os
import pickle
from sklearn.datasets import fetch_20newsgroups

categories = ['sci.med', 'talk.politics.misc',  'rec.autos']
data = fetch_20newsgroups(subset='train', categories=categories,remove=('headers', 'footers', 'quotes'))

dst_name = "20newsgroup.pckl"
dst_path = os.path.join("data", dst_name)
with open(dst_path, 'wb') as fout:
    pickle.dump(data, fout)

In [2]:
from pprint import pprint

newsgroups_train = fetch_20newsgroups(subset='train')
pprint(list(newsgroups_train.target_names))


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [3]:
# loada data pickle
with open(dst_path, 'rb') as fin:
    data = pickle.load(fin)

data = [doc for doc in data.data]

In [4]:
data[6]

'\nOk boys and girls,\n\n"What was the \'Ogadan War\'????"\n\nThe Money Raised in Band-Aid covered How Much of\nthe Cost of Which Soviet Client State to replace what\ncatagory of weapon system lost in the aforementioned war?\n\nWhy was the Joke: "We arm the World." Really Not that funny?\n\nGonzo Station is the designation for WHICH USN Op Area?\nand the primary threat targets in the Area Were:.....\n\nciao\ndrieux\n\n\n'

In [5]:
# preprocessing
import re
from nltk import sent_tokenize
from nltk import word_tokenize

def preprocess(doc):
    sents = sent_tokenize(doc)
    sents_tok = list() # tokenisasi kalimat
    for s in sents:
        s = s.strip().lower() # case folding dan menghilangkan new line
        s = s.replace("\n", " ") # menggantikan \n dengan spasi
        s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s) # menghapus simbol
        s = re.sub(' +', ' ', s) # menghapus repetitive space
        
        sents_tok.append(s)
    
    return " ".join(sents_tok)

docs = list()
for d in data:
    docs.append(preprocess(d))
#docs

In [6]:
# Untuk membentuk word embedding digunakan library Gensim
# Format inputan dari Gensim adalah list of words untuk setiap kalimat
# Contoh pada dokumen:
#       Tiger is the biggest cat alive. The tigers species can be divided into 5 groups.
# Inputan yang dibutuhkan Gensim adalah:
#       ["tiger", "is", "the", "biggest", "cat", "alive"], ["the", "tigers", "species", 
#        "can", "be", "devided", "into", "5", "groups"]
#
doc_gensim = [word_tokenize(d) for d in docs]
#print(doc_gensim)

In [7]:
# train skip-gram dengan library gensim
#
from gensim.models import Word2Vec

d = 300
model_sg = Word2Vec(doc_gensim, min_count=1, size=d, window=5, workers=-1)
print("DONE")

DONE


In [8]:
import os

model_path = os.path.join('model', 'model_sg.model')
# model dari word embedding dapat disimpan dengan fungsi di bawah ini
model_sg.save(model_path)

# untuk menggunakan model yang telah disimpan sebelumnya dapat dilakukan
# dengan memanfaatkan fungsi load, beberapa model membutuhkan waktu yang lama
# karena memiliki ukuran yang besar
model_sg_load = Word2Vec.load(model_path)

print('DONE')


DONE


#### Word2 Vec menggunakan matriks dense

<p>Penggunaan memori oleh Gensim</p>
<p>Jumlah kata x size x 12 bytes <br>
    Contoh: <br>
    Jika terdapat 100.000 kata unik dengan menggunakan dimensi embedding 200 <br>
    Maka memori yang dibutuhkan = 100.000 x 200 x 12 bytes = ~229MB
</p>

In [9]:
# Melihat vektor suatu kata
model_sg.wv['car']

array([-1.23108237e-03, -8.02676484e-04, -3.23985354e-04,  1.28359965e-03,
       -1.23454630e-03,  2.43231596e-04, -1.51430408e-03,  1.23896624e-03,
       -1.36310980e-03,  4.81080526e-04,  3.91767389e-04, -5.23407303e-04,
       -1.00291264e-03, -7.79745460e-04,  1.54002185e-03, -9.50106012e-04,
       -1.54890309e-04,  7.70264978e-06,  5.63529786e-04, -5.98794490e-04,
        1.22962752e-03, -1.62316801e-03,  2.40724316e-04,  9.46418557e-04,
        9.41080682e-04, -3.28972877e-04, -1.00694376e-03, -4.19931021e-04,
        4.36858652e-04,  1.74907182e-04, -1.18424003e-04,  4.89391328e-04,
        3.42271611e-04,  1.64676784e-03, -1.60131755e-03, -3.01632303e-04,
       -5.26122807e-04, -1.00837671e-03,  4.58112685e-04, -6.43660096e-05,
        9.44961794e-04,  7.14395079e-04,  5.05643548e-04,  1.10773498e-03,
       -9.45286010e-04,  1.43273245e-03, -2.21142880e-04, -3.54002899e-04,
       -1.17031636e-03,  9.68187058e-04,  1.33653020e-03, -1.07109104e-03,
        1.50150037e-03, -

In [22]:
# Menghitung similarity vektor antar kata

model_sg.wv.similarity('car', 'car')

1.0

In [11]:
#  Menampilkan top-N similar words

model_sg.wv.similar_by_word('car', topn=10)

[('millions', 0.23960770666599274),
 ('tore', 0.23730003833770752),
 ('responces', 0.2234179526567459),
 ('diab', 0.22310039401054382),
 ('hp', 0.22308149933815002),
 ('indoctrinated', 0.2147151529788971),
 ('minimal', 0.20774942636489868),
 ('startrek', 0.20443326234817505),
 ('hyper', 0.19632530212402344),
 ('hitting', 0.19322627782821655)]

In [12]:
# Mencari yang paling mirip dengan kata 'cars'
model_sg.wv.most_similar('cars')

[('gar', 0.23239296674728394),
 ('wnbc', 0.2048788070678711),
 ('gloom', 0.19768092036247253),
 ('floation', 0.19452424347400665),
 ('crank', 0.19183796644210815),
 ('honistly', 0.191462904214859),
 ('winegar', 0.1896638125181198),
 ('urologie', 0.1883377730846405),
 ('neurological', 0.18824419379234314),
 ('prostate', 0.18742118775844574)]

<p><img alt="" src="figures/3_cosine.png" style="height:400px; width:683px" /></p>

In [13]:
# error jika kata tidak ada di training data

kata = 'entir'
try:
    print(model_sg.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')

error! kata " entir " tidak ada di training data


### Note: ###
<ul>
    <li> Pada word2vec apabila suatu kata tidak terdapat dalam list vocabulary maka akan error </li>
    <li> Kesalahan pengetikan (typo) bisa menimbulkan error </li>
    <li> Untuk mangatasi hal ini dapat dilakukan dengan menggunakan algoritma lain, seperti FastText </li>

In [14]:
# "predict" vector for new data without re-training from the beginning
d1 = ['new','generation','nvidia','gpu','is','rtx']
d2 = ['deep','learning','computation','mostly', 'on', 'gpu']
d3 = ['the','rtx','gpu','capable','super','sampling','ssdl']
D = [d1,d2,d3]
model_sg.train(D, total_examples=len(D), epochs=model_sg.epochs)
print(model_sg.predict_output_word('gpu'))

[('intrigues', 4.4517652e-05), ('enduring', 4.4517652e-05), ('cardinal', 4.4517652e-05), ('shirer', 4.4517652e-05), ('chancellors', 4.4517652e-05), ('undermining', 4.4517652e-05), ('waning', 4.4517652e-05), ('strasser', 4.4517652e-05), ('schleicher', 4.4517652e-05), ('unite', 4.4517652e-05)]


<h3 id="Latihan:"><font color="blue">Latihan 1:</font></h3>

<ul> 
    <li> Bangunlah word embedding dengan data 3 kategori lainnya dari data 20newsGroup</li>
    <li> Lakukan training menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah vektor yang dihasilkan oleh suatu kata sama? ketika menggunakan Skip-Gram dan CBoW </li>
    <li> Apakah hasil top-n similar memperoleh vocabulary yang sama? </li>
    <li> Apakah jumlah dimensi berpengaruh pada hasil top-n most similar? </li>

In [15]:
# Kerjakan latihan 1 pada cell berikut ini


## FastText (Facebook-2016)##
<ul>
    <li> Menggunakan Sub-words: app, ppl, ple - apple, sehingga dapat mengatasi permasalahan typo atau tidak adanya suatu kata dalam vocabularies </li>
    <li> Paper: https://arxiv.org/abs/1607.04606  </li>
    <li> Website: https://fasttext.cc/ </li>
    <li> Source: https://github.com/facebookresearch/fastText </li>
</ul>

In [16]:
from gensim.models import FastText

dim = 300 # Jumlah neurons = ukuran vektor = jumlah kolom
model_ft = FastText(doc_gensim, size=dim, window=5, min_count=2, workers=-1)
'Done'

'Done'

In [17]:
#  Menampilkan top-N similar words

model_ft.wv.similar_by_word('car', topn=10)

[('cart', 0.3586006462574005),
 ('carb', 0.34631019830703735),
 ('cars', 0.32502445578575134),
 ('carbs', 0.3238670527935028),
 ('carter', 0.3183533549308777),
 ('cards', 0.30618295073509216),
 ('cartel', 0.30090874433517456),
 ('carr', 0.2989318370819092),
 ('card', 0.2915874421596527),
 ('scar', 0.28857722878456116)]

In [18]:
# Mencari yang paling mirip dengan kata 'cars'
model_ft.wv.most_similar('cars')

[('carriers', 0.32694974541664124),
 ('car', 0.32502445578575134),
 ('stars', 0.32168301939964294),
 ('card', 0.3070753812789917),
 ('ears', 0.28709676861763),
 ('carr', 0.27693971991539),
 ('islanders', 0.27515748143196106),
 ('carb', 0.2651306092739105),
 ('years', 0.2617567479610443),
 ('gears', 0.26143354177474976)]

In [19]:
# Melihat similarity antar kata

print(model_ft.wv.similarity('cars', 'car'))
print(model_sg.wv.similarity('cars', 'car'))

0.32502446
-0.008061564


In [20]:
# Tidak error jika kata tidak ada di training data

kata = 'beckmans'
try:
    print(model_ft.wv.most_similar(kata))
except:
    print('error! kata "',kata,'" tidak ada di training data')
    

# Tidak terjadi error saat kata tidak terdapat pada vocabulary

[('germans', 0.29016056656837463), ('humans', 0.2773742973804474), ('lemans', 0.24586813151836395), ('fans', 0.23251758515834808), ('loans', 0.22750471532344818), ('vans', 0.21468831598758698), ('orwell', 0.21389256417751312), ('thier', 0.20579759776592255), ('teens', 0.20407643914222717), ('evans', 0.20316636562347412)]


<h3 id="Latihan:"><font color="blue">Latihan 2:</font></h3>

<ul>
	<li>Apakah kelebihan dan kekurangan WE secara umum?</li>
	<li>Apakah kira-kira aplikasi yang dapat memanfaatkan WE?</li>
	<li>Apakah bisa dijadikan representasi dokumen? Bagaimana caranya?</li>
	<li>Bergantung pada apa sajakah performa model WE?</li>
</ul>

* Preprocessing apa yang sebaiknya dilakukan pada model Word Embedding?
* Apakah Pos Tag bermanfaat disini? Jika iya bagaimana menggunakannya?

In [21]:
# Kerjakan latihan 2 pada cell berikut ini